# Nashville Processor

- Remove all rows containing a missing value in a mandatory column
- Remove columns ‘image’, ‘Sold As Vacant’ and ‘Multiple Parcels Involved in Sale’
- Add columns:
   - Price per square foot
   - Age of property
   - Sale year and sale month
   - Land-to-building value ratio
   - Sale price category: Low (< 100 000), Medium (100 000- 300 000), High (>300 000)
   - Family Name and First name of owner (first person listed)

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import os
from reader import Reader


In [ ]:
class Processor:
    
    def __init__(self):
        reader = Reader()
        self.data = reader.load_data()
    
    # Remove all rows containing a missing value in a mandatory column.    
    def remove_rows_with_missing_mandatory_values(self):
        self.data = self.data.dropna(subset=['Land Value'])
        return self
        
    # Remove columns ‘image’, ‘Sold As Vacant’ and ‘Multiple Parcels Involved in Sale’.
    def remove_columns(self):
        columns_to_remove = ['image', 'Sold As Vacant', 'Multiple Parcels Involved in Sale']
        existing_columns = [col for col in columns_to_remove if col in self.data.columns]
        self.data = self.data.drop(columns=existing_columns)
        return self
        
    # Price per square foot.
    def add_price_per_sqft(self):
        self.data['Price per Square Foot'] = self.data['Sale Price'] / self.data['Finished Area']
        return self
    
    # Age of property.
    def add_property_age(self):
        if not pd.api.types.is_datetime64_any_dtype(self.data['Sale Date']):
            self.data['Sale Date'] = pd.to_datetime(self.data['Sale Date'])
            
        sale_years = self.data['Sale Date'].dt.year
        
        self.data['Property Age'] = sale_years - self.data['Year Built']
        return self
        
    # Sale year and sale month.
    def add_sale_year_month(self):
        if not pd.api.types.is_datetime64_any_dtype(self.data['Sale Date']):
            self.data['Sale Date'] = pd.to_datetime(self.data['Sale Date'])
            
        self.data['Sale Year'] = self.data['Sale Date'].dt.year
        self.data['Sale Month'] = self.data['Sale Date'].dt.month
        return self
        
    # Land-to-building value ratio.
    def add_land_building_ratio(self):
        self.data['Land-to-Building Ratio'] = self.data['Land Value'] / self.data['Building Value'].replace(0, np.nan)
        return self
     
    # Sale price category: Low (< 100 000), Medium (100 000 - 300 000), High (>300 000).   
    def add_price_category(self):
        conditions = [
            self.data['Sale Price'] < 100000,
            (self.data['Sale Price'] >= 100000) & (self.data['Sale Price'] <= 300000),
            self.data['Sale Price'] > 300000
        ]
        choices = ['Low', 'Medium', 'High']
        self.data['Sale Price Category'] = np.select(conditions, choices, default='Unknown')
        return self
    
    
    # Family Name and First name of owner (first person listed)
    def extract_owner_names(self):
        def extract_names(owner_string):
            if pd.isna(owner_string):
                return pd.Series([np.nan, np.nan])
                
            owner_string = str(owner_string).strip()
            
            if '&' in owner_string:
                owner_string = owner_string.split('&')[0].strip()
            if ',' in owner_string:
                parts = owner_string.split(',', 1)
                family_name = parts[0].strip()
                first_name = parts[1].strip() if len(parts) > 1 else np.nan
            else:
                parts = owner_string.split()
                if len(parts) > 1:
                    first_name = parts[0].strip()
                    family_name = ' '.join(parts[1:]).strip()
                else:
                    family_name = owner_string
                    first_name = np.nan
                    
            return pd.Series([family_name, first_name])
            
        self.data[['Family Name', 'First Name']] = self.data['Owner Name'].apply(extract_names)
        return self
    
    # Process all the clean up functions.
    def process(self):
        self.remove_rows_with_missing_mandatory_values()
        self.remove_columns()
        self.add_price_per_sqft()
        self.add_property_age()
        self.add_sale_year_month()
        self.add_land_building_ratio()
        self.add_price_category()
        self.extract_owner_names()
        self.data = self.data.reset_index(drop=True)
        return self
    
    #Save the data to the /output folder.
    def save_data(self):
        output_path = 'output/processed_nashville_housing.csv'
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        self.data.to_csv(output_path, index=False)
        return self
    
    # To show the processed data.   
    def get_processed_data(self):
        return self.data

In [3]:
processor = Processor()
processor.process()
processor.save_data()
processed_data = processor.get_processed_data()
processed_data

,Unnamed: 0.1,Unnamed: 0,Parcel ID,Land Use,Property Address,Suite/ Condo #,Property City,Sale Date,Sale Price,Legal Reference,Owner Name,Address,City,State,Acreage,Tax District,Neighborhood,Land Value,Building Value,Total Value,Finished Area,Foundation Type,Year Built,Exterior Wall,Grade,Bedrooms,Full Bath,Half Bath,Price per Square Foot,Property Age,Sale Year,Sale Month,Land-to-Building Ratio,Sale Price Category,Family Name,First Name
0,1,1,105 11 0 080.00,SINGLE FAMILY,1802 STEWART PL,NaN,NASHVILLE,2013-01-11,191500,20130118-0006337,"STINSON, LAURA M.",1802 STEWART PL,NASHVILLE,TN,0.17,URBAN SERVICES DISTRICT,3127.0,32000.0,134400.0,168300.0,1149.00000,PT BSMT,1941.0,BRICK,C,2.0,1.0,0.0,166.666667,72.0,2013,1,0.238095,Medium,STINSON,LAURA M.
1,2,2,118 03 0 130.00,SINGLE FAMILY,2761 ROSEDALE PL,NaN,NASHVILLE,2013-01-18,202000,20130124-0008033,"NUNES, JARED R.",2761 ROSEDALE PL,NASHVILLE,TN,0.11,CITY OF BERRY HILL,9126.0,34000.0,157800.0,191800.0,2090.82495,SLAB,2000.0,BRICK/FRAME,C,3.0,2.0,1.0,96.612583,13.0,2013,1,0.215463,Medium,NUNES,JARED R.
2,3,3,119 01 0 479.00,SINGLE FAMILY,224 PEACHTREE ST,NaN,NASHVILLE,2013-01-18,32000,20130128-0008863,"WHITFORD, KAREN",224 PEACHTREE ST,NASHVILLE,TN,0.17,URBAN SERVICES DISTRICT,3130.0,25000.0,243700.0,268700.0,2145.60001,FULL BSMT,1948.0,BRICK/FRAME,B,4.0,2.0,0.0,14.914243,65.0,2013,1,0.102585,Low,WHITFORD,KAREN
3,4,4,119 05 0 186.00,SINGLE FAMILY,316 LUTIE ST,NaN,NASHVILLE,2013-01-23,102000,20130131-0009929,"HENDERSON, JAMES P. & LYNN P.",316 LUTIE ST,NASHVILLE,TN,0.34,URBAN SERVICES DISTRICT,3130.0,25000.0,138100.0,164800.0,1969.00000,CRAWL,1910.0,FRAME,C,2.0,1.0,0.0,51.802946,103.0,2013,1,0.181028,Medium,HENDERSON,JAMES P.
4,5,5,119 05 0 387.00,SINGLE FAMILY,2626 FOSTER AVE,NaN,NASHVILLE,2013-01-04,93736,20130118-0006110,"MILLER, JORDAN",2626 FOSTER AVE,NASHVILLE,TN,0.17,URBAN SERVICES DISTRICT,3130.0,25000.0,86100.0,113300.0,1037.00000,CRAWL,1945.0,FRAME,C,2.0,1.0,0.0,90.391514,68.0,2013,1,0.290360,Low,MILLER,JORDAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26012,56607,56607,176 09 0 003.00,SINGLE FAMILY,4964 HICKORY WOODS E,NaN,ANTIOCH,2016-10-28,236000,20161031-0114817,"CHHAY, CHOWAN & NIM, PHALLY",4964 HICKORY WOODS E,ANTIOCH,TN,0.23,URBAN SERVICES DISTRICT,6328.0,25000.0,159300.0,184300.0,3117.00000,SLAB,1995.0,BRICK/FRAME,C,3.0,3.0,0.0,75.713827,21.0,2016,10,0.156937,Medium,CHHAY,CHOWAN
26013,56612,56612,081 16 0 197.00,VACANT RESIDENTIAL LAND,1017 MONROE ST,NaN,NASHVILLE,2016-10-24,231500,20161026-0113388,"ZHANG, JUN YEN",1017 MONROE ST,NASHVILLE,TN,0.17,URBAN SERVICES DISTRICT,126.0,40000.0,0.0,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,10,NaN,Medium,ZHANG,JUN YEN
26014,56614,56614,082 05 0 040.00,SINGLE FAMILY,1625 5TH AVE N,NaN,NASHVILLE,2016-10-28,466000,20161102-0115988,"GLAUS, WILLIAM D. SR.",1625 5TH AVE N,NASHVILLE,TN,0.15,URBAN SERVICES DISTRICT,126.0,40000.0,204100.0,256000.0,1637.00000,CRAWL,2004.0,FRAME,B,3.0,2.0,1.0,284.667074,12.0,2016,10,0.195982,High,GLAUS,WILLIAM D. SR.
26015,56615,56615,082 05 0 058.00,SINGLE FAMILY,1614 5TH AVE N,NaN,NASHVILLE,2016-10-26,685000,20161101-0115366,"DUNN, JEFFREY J. & HOWE, TRICIA L.",1614 5TH AVE N,NASHVILLE,TN,0.19,URBAN SERVICES DISTRICT,126.0,40000.0,295900.0,351600.0,2478.00000,CRAWL,2005.0,FRAME,B,4.0,3.0,1.0,276.432607,11.0,2016,10,0.135181,High,DUNN,JEFFREY J.
